In [2]:
import os
from langchain_core.prompts import ChatPromptTemplate
import re
import json

In [4]:
from google import genai

client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Explain how AI works in a few words"
)
print(response.text)

AI learns patterns from data to make predictions or decisions.


```
Input               Agent                     Output
Query           -> Filter ->      Từ khoá (Subject, chapter, ...)
Query + Chunks  -> LLMs trả lời ->          Kết quả cuối cùng
```

## ***Prompt dùng để trả về từ khoá để thực hiện metadata filtering***

In [ ]:
""" 
        Mày là một con Agent sẽ thực hiện filter các từ khoá quan trọng thông qua các từ khoá quy định sau đây.
        
        Các từ khoá quy định: 
            subject, chapter, chapter_title, section, section_title, subsection, subsection_title, type.
        
        Chapter
        Section
        Type
        
        Chi tiết các từ khoá:
            - Subject: Tên môn học, ví dụ: "Lịch sử Đảng Cộng sản Việt Nam"
            - Chapter: Chương của môn học, ví dụ: "Chương 1"
            - Chapter title: Tiêu đề của chương, ví dụ: "Lịch sử Đảng Cộng sản Việt Nam"
            - Section: Phần của chương, ví dụ: "I" hoặc "1"
            - Section title: Tiêu đề của phần, ví dụ: "Nguyên nhân hình thành Đảng Cộng sản Việt Nam" hoặc "1. Nguyên nhân hình thành Đảng Cộng sản Việt Nam"
            - Sub-section: Tiểu mục trong phần, ví dụ: "1." hoặc "2."
            - Sub-section title: Tiêu đề của tiểu mục, ví dụ: "Nguyên nhân hình thành Đảng Cộng sản Việt Nam" hoặc "Lịch sử Đảng Cộng sản Việt Nam"
            - Type: Phân loại nội dung, sẽ có 3 kiểu "TARGET", "EXERCISES", "THEORY". Trong đó:
                + "TARGET": Là những nội dung chính của chương, phần, tiểu mục.
                + "EXERCISES": Là những bài tập, câu hỏi liên quan đến chương
                + "THEORY": Là những lý thuyết, kiến thức nền tảng liên quan đến chương, phần, tiểu mục.

        Đây là câu query của người dùng: 
        {query}
        
        ** Lưu ý:
        - Chỉ trả về các từ khoá quan trọng theo định dạng json. Kết quả trả về sẽ là một đối tượng JSON với các trường tương ứng với các từ khoá đã nêu ở trên. Không bao gồm những nội dung không cần thiết.
        - Ví dụ kết quả trả về:
        {{
            "subject": {{"$eq": "Lịch sử Đảng Cộng sản Việt Nam"}},
            "chapter_title": {{"$eq": "sự hình thành của Đảng Cộng sản Việt Nam"}},
            "section_title": {{"$eq": "sự hình thành của Đảng Cộng sản Việt Nam"}},
            "subsection_title": {{"$eq": "sự hình thành của Đảng Cộng sản Việt Nam"}},
            "type": {{"$eq": "THEORY"}}
        }}
        - Với các từ khoá trên, với mỗi từ khoá tương ứng, nếu không tìm được thông tin thì không cần trả về từ khoá.
    """
    
    """
    - Subject: Tên môn học, ví dụ: "Lịch sử Đảng Cộng sản Việt Nam". Subject sẽ là tên của một trong 3 môn học sau:
                + "Lịch sử Đảng Cộng sản Việt Nam"
                + "Triết học Mác - Lênin"
                + "Kinh tế chính trị Mác - Lênin"
    """
                "subject": {{"$eq": "Lịch sử Đảng Cộng sản Việt Nam"}},

In [26]:
def get_keywords(query):
    system_prompt =""" 
        Mày là một con Agent sẽ thực hiện filter các từ khoá quan trọng thông qua các từ khoá quy định sau đây.
        
        Các từ khoá quy định: 
            type.
        
        Chi tiết các từ khoá:
            - Type: Phân loại nội dung, sẽ có 3 kiểu "TARGET", "EXERCISES", "THEORY". Trong đó:
                + "TARGET": Là những nội dung chính của chương, phần, tiểu mục.
                + "EXERCISES": Là những bài tập, câu hỏi liên quan đến chương
                + "THEORY": Là những lý thuyết, kiến thức nền tảng liên quan đến chương, phần, tiểu mục.

        Đây là câu query của người dùng: 
        {query}
        
        ** Lưu ý:
        - Chỉ trả về các từ khoá quan trọng theo định dạng json. Kết quả trả về sẽ là một đối tượng JSON với các trường tương ứng với các từ khoá đã nêu ở trên. Không bao gồm những nội dung không cần thiết.
        - Ví dụ kết quả trả về:
        {{
            "type": {{"$eq": "THEORY"}}
        }}
        - Với các từ khoá trên, với mỗi từ khoá tương ứng, nếu không tìm được thông tin thì không cần trả về từ khoá.
    """

    prompt_template = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
    ])

    messages = prompt_template.format_messages(
        query=query.strip()
    )
    # "Chương nào của môn Lịch sử Đảng Cộng sản Việt Nam có nội dung về sự hình thành của Đảng Cộng sản Việt Nam?"
    response = client.models.generate_content(
        model="gemini-2.5-flash", 
        contents=messages[0].content.strip()
    )
    
    clean_text = re.sub(r"```json|```", "", response.text.strip()).strip()
    parsed = json.loads(clean_text)
    return parsed

## ***Model embedding***

In [13]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("AITeamVN/Vietnamese_Embedding")

## ***Gọi database***

In [21]:
query = "Cho tôi một vài câu hỏi luyện tập môn lịch sử đảng của chương 2"
keywords = get_keywords(query)
keywords

{'subject': {'$eq': 'Lịch sử Đảng Cộng sản Việt Nam'},
 'type': {'$eq': 'EXERCISES'}}

In [15]:
test = {'subject': {'$eq': ''},
 'type': {'$eq': ''}}

In [33]:
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_key)

# To get the unique host for an index, 
# see https://docs.pinecone.io/guides/manage-data/target-an-index
index = pc.Index(host="https://rag-ktcn-0abpzwm.svc.aped-4627-b74a.pinecone.io")

query = ""
embed_query = embedding_model.encode(query, convert_to_tensor=False).tolist()
keywords = get_keywords(query)
print(keywords)
results = index.query(
    namespace="lich-su-dang", 
    vector=embed_query,
    top_k=5,
    filter={
        "chapter": {"$eq": "Chương 2"}},  # test,
    include_metadata=True
    # fields=["category", "chunk_text"]
)
print(results)
# embed_query

{}
{'matches': [{'id': 'LSD_chuong2_I_2_6',
              'metadata': {'chapter': 'Chương 2',
                           'chapter_title': 'ĐẢNG LÃNH ĐẠO HAI CUỘC KHÁNG '
                                            'CHIẾN, HOÀN THÀNH GIẢI PHÓNG DÂN '
                                            'TỘC, THỐNG NHẤT ĐẤT NƯỚC (1945 - '
                                            '1975)',
                           'content': '. Lực lượng công an được thống nhất tổ '
                                      'chức trong toàn quốc và hoạt động cả '
                                      'trong vùng địch hậu và vùng tự do. Tổ '
                                      'chức của Nha Công an Việt Nam được '
                                      'thiết lập theo hệ thống dọc đến cấp '
                                      'Khu, Ty, Quận, huyện, có sự phát triển '
                                      'mới cả về biên chế, tổ chức bộ máy, lý '
                                      'luận nghiệp vụ. Công an nhân